In [13]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import *
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python import rl_environment


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [14]:
game_name = 'large_game_2'
game_config = json.load(open(f"{os.environ['CLOCK_AUCTION_CONFIG_DIR']}/{game_name}.json"))
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename=f'{game_name}.json'))


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


In [15]:
x = np.array(game.new_initial_state().child(0).child(0).child(10).child(10).child(10).child(10).information_state_tensor())
x[x.nonzero()]

array([ 1.00000000e+00,  1.00000000e+00,  3.00000000e+00, -2.50000000e-01,
       -5.00000000e-01, -7.50000000e-01,  1.95000000e+01,  1.92500000e+01,
        1.90000000e+01,  1.87500000e+01,  3.90000000e+01,  3.87500000e+01,
        3.85000000e+01,  3.82500000e+01,  5.85000000e+01,  5.82500000e+01,
        5.80000000e+01,  5.77500000e+01,  4.50000000e+01,  4.47500000e+01,
        4.45000000e+01,  4.42500000e+01,  6.45000000e+01,  6.42500000e+01,
        6.40000000e+01,  6.37500000e+01,  8.40000000e+01,  8.37500000e+01,
        8.35000000e+01,  8.32500000e+01,  1.03500000e+02,  1.03250000e+02,
        1.03000000e+02,  1.02750000e+02,  9.00000000e+01,  8.97500000e+01,
        8.95000000e+01,  8.92500000e+01,  1.09500000e+02,  1.09250000e+02,
        1.09000000e+02,  1.08750000e+02,  1.29000000e+02,  1.28750000e+02,
        1.28500000e+02,  1.28250000e+02,  1.48500000e+02,  1.48250000e+02,
        1.48000000e+02,  1.47750000e+02,  1.35000000e+02,  1.34750000e+02,
        1.34500000e+02,  

In [16]:
env = rl_environment.Environment(game)
num_actions = env.action_spec()["num_actions"]
time_step = env.reset()

agents = [TakeSingleActionDecorator(StraightforwardAgent(i, game_config, num_actions), num_actions) for i in range(game.num_players())]

while not time_step.last():
    player_id = time_step.observations["current_player"]
    agent = agents[player_id]
    agent_output = agent.step(time_step)
    print(player_id, agent_output.action)
    action_list = [agent_output.action]
    time_step = env.step(action_list)

for i, agent in enumerate(agents):
    agent.step(time_step)


0 63
1 63
0 63
1 63
0 47
1 47
0 3
1 15
0 3
1 15
0 3
1 15
0 3
1 15
0 0
1 15
